# Lab (3) Advanced CNN architectures
 Implement the ResNet-20 architecture

In [1]:
# import necessary dependencies
import argparse
import os, sys
import time
import datetime
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [2]:
# create custom ResBlock
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample=False):
        super(ResBlock, self).__init__()
        if downsample == False:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.skip = nn.Sequential()          
        else:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
                nn.BatchNorm2d(out_channels)
            )

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.conv1_bn = nn.BatchNorm2d(out_channels)
        self.conv2_bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        skip = self.skip(x)
        out = self.conv1(x)
        out = F.relu(self.conv1_bn(out))
        out = self.conv2(out)
        out = F.relu(self.conv2_bn(out))
        out = out + skip
        out = F.relu(out)
        return out

In [3]:
# define ResNet model;
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv = nn.Conv2d(3, out_channels = 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv_bn  = nn.BatchNorm2d(16)

        self.res1 = nn.Sequential(
            ResBlock(16, 16),
            ResBlock(16, 16),
            ResBlock(16, 16)
        )

        self.res2 = nn.Sequential(
            ResBlock(16, 32, downsample=True),
            ResBlock(32, 32, downsample=False),
            ResBlock(32, 32, downsample=False)
        )

        self.res3 = nn.Sequential(
            ResBlock(32, 64, downsample=True),
            ResBlock(64, 64, downsample=False),
            ResBlock(64, 64, downsample=False)
        )
        
        self.fc = nn.Linear(64, 10)

    def forward(self, x):
        out = self.conv(x)
        #out = F.max_pool2d(out, kernel_size = 3, stride = 2, padding = 1)
        out = F.relu(self.conv_bn(out))
        out = self.res1(out)
        out = self.res2(out)
        out = self.res3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [4]:
#############################################
# sanity check for the correctness of SimpleNN
net = ResNet()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net.to(device)
#############################################

ResNet(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (res1): Sequential(
    (0): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (skip): Sequential()
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv1_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (skip): Sequential()
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv1_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, tr

In [5]:
print(net)

ResNet(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (res1): Sequential(
    (0): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (skip): Sequential()
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv1_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (skip): Sequential()
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv1_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, tr

## Step 1: Set up preprocessing functions
Preprocessing is very important as discussed in the lecture.
You will need to write preprocessing functions with the help of *torchvision.transforms* in this step.
You can find helpful tutorial/API at [here](https://pytorch.org/vision/stable/transforms.html).

### Question (b)
For the question, you need to:
1. Complete the preprocessing code below.
2. **In the PDF report**, briefly describe what preprocessing operations you used and what are the purposes of them.

Hint: 
1. Only two operations are necessary to complete the basic preprocessing here.
2. The raw input read from the dataset will be PIL images.
3. Data augmentation operations are not mendatory, but feel free to incorporate them if you want.
4. Reference value for mean/std of CIFAR-10 images (assuming the pixel values are within [0,1]): mean (RGB-format): (0.4914, 0.4822, 0.4465), std (RGB-format): (0.2023, 0.1994, 0.2010)

In [6]:
# useful libraries
import torchvision
import torchvision.transforms as transforms

#############################################
# your code here
# specify preprocessing function
# convert it to a tensor, and normalize
# add random crop with padding of 4 and random flips to image
transform_train = transforms.Compose([
    transforms.RandomCrop(size=(32, 32), padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994,0.2010))])

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994,0.2010))])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994,0.2010))])
#############################################

## Step 2: Set up dataset and dataloader

### Question (c)
Set up the train/val datasets and dataloders that are to be used during the training. Check out the [official API](https://pytorch.org/docs/stable/data.html) for more information about **torch.utils.data.DataLoader**.

Here, you need to:
1. Complete the code below.

In [7]:
# do NOT change these
from tools.dataset import CIFAR10
from torch.utils.data import DataLoader

torch.manual_seed(0)

# a few arguments, do NOT change these
DATA_ROOT = "./data"
TRAIN_BATCH_SIZE = 128
VAL_BATCH_SIZE = 100

#############################################
# your code here
# construct dataset
train_set = CIFAR10(
    root=DATA_ROOT, 
    mode='train', 
    download=True,
    transform=transform_train    # your code
)
val_set = CIFAR10(
    root=DATA_ROOT, 
    mode='val', 
    download=True,
    transform=transform_val   # your code
)
test_set = CIFAR10(
    root=DATA_ROOT, 
    mode='test', 
    download=True,
    transform=transform_test   # your code
)

# construct dataloader
train_loader = DataLoader(
    train_set, 
    batch_size=TRAIN_BATCH_SIZE,  # your code
    shuffle=True,     # your code
    num_workers=2
)
val_loader = DataLoader(
    val_set, 
    batch_size=VAL_BATCH_SIZE,  # your code
    shuffle=False,     # your code
    num_workers=2
)
test_loader = DataLoader(
    test_set, 
    batch_size=100,  # your code
    shuffle=False,     # your code
    num_workers=2
)
#############################################

Using downloaded and verified file: ./data/cifar10_trainval_F22.zip
Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified
Using downloaded and verified file: ./data/cifar10_trainval_F22.zip
Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified
Using downloaded and verified file: ./data/cifar10_test_F22.zip
Extracting ./data/cifar10_test_F22.zip to ./data
Files already downloaded and verified


## Step 3: Instantiate your SimpleNN model and deploy it to GPU devices.
### Question (d)
You may want to deploy your model to GPU device for efficient training. Please assign your model to GPU if possible. If you are training on a machine without GPUs, please deploy your model to CPUs.

Here, you need to:
1. Complete the code below.
2. **In the PDF report**, briefly describe how you verify that your model is indeed deployed on GPU. (Hint: check $\texttt{nvidia-smi}$.)

In [8]:
# specify the device for computation
#############################################
# your code here
if torch.cuda.is_available():
    # a CUDA device object
    device = torch.device("cuda")          
    # casting tensors to a cuda data type
    dtype = torch.cuda.FloatTensor  
    print(torch.cuda.is_available())
    print(torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    
#############################################

True
Tesla T4


## Step 4: Set up the loss function and optimizer
Loss function/objective function is used to provide "feedback" for the neural networks. Typically, we use multi-class cross-entropy as the loss function for classification models. As for the optimizer, we will use SGD with momentum. 

### Question (e)
Here, you need to:
1. Set up the cross-entropy loss as the criterion. (Hint: there are implemented functions in **torch.nn**)
2. Specify a SGD optimizer with momentum. (Hint: there are implemented functions in **torch.optim**)

In [9]:
import torch.nn as nn
import torch.optim as optim

net = ResNet()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net.to(device)

# hyperparameters
# initial learning rate
INITIAL_LR = 0.1

# momentum for optimizer
MOMENTUM = 0.9

# L2 regularization strength
REG = 0.0005

#############################################
# your code here
# create loss function
criterion = nn.CrossEntropyLoss()

# Add optimizer
optimizer = torch.optim.SGD(params = net.parameters(), lr=INITIAL_LR, momentum = MOMENTUM, nesterov=True, weight_decay = REG)
#############################################

## Step 5: Start the training process.

### Question (f)/(g)
Congratulations! You have completed all of the previous steps and it is time to train our neural network.

Here you need to:
1. Complete the training codes.
2. Actually perform the training.

Hint: Training a neural network usually repeats the following 4 steps: 

**i) Get a batch of data from the dataloader and copy it to your device (GPU).**

**ii) Do a forward pass to get the outputs from the neural network and compute the loss. Be careful about your inputs to the loss function. Are the inputs required to be the logits or softmax probabilities?)**

**iii) Do a backward pass (back-propagation) to compute gradients of all weights with respect to the loss.**

**iiii) Update the model weights with the optimizer.**

You will also need to compute the accuracy of training/validation samples to track your model's performance over each epoch (the accuracy should be increasing as you train for more and more epochs).


In [10]:
import math
import matplotlib.pyplot as plt

In [11]:
EPOCHS = 200

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 150], gamma=0.1)

# the folder where the trained model is saved
CHECKPOINT_FOLDER = "./saved_model"

lrs = []

# start the training/validation process
# the process should take about 5 minutes on a GTX 1070-Ti
# if the code is written efficiently.
best_val_acc = 0

print("==> Training starts!")
print("="*50)

for i in range(0, EPOCHS):
    # handle the learning rate scheduler.
    #######################
    # your code here
    # switch to train mode
    net.train()
    #######################
    print("Epoch %d:" %i, 'LR:', scheduler.get_lr())
    # this help you compute the training accuracy
    total_examples = 0
    correct_examples = 0

    train_loss = 0 # track training loss if you want
    
    # Train the model for 1 epoch.
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        ####################################
        # your code here
        #print('Epoch {}, Batch idx {}, lr {}'.format(EPOCHS, batch_idx, optimizer.param_groups[0]['lr']))
        # copy inputs to device
        inputs, targets = inputs.to(device), targets.to(device) 
        
        # compute the output and loss
        y_pred = net(inputs)
        loss = criterion(y_pred, targets)
        
        # zero the gradient
        optimizer.zero_grad()
        
        # backpropagation
        loss.backward()
        train_loss += loss.item()
                         
        # apply gradient and update the weights
        optimizer.step()

        # count the number of correctly predicted samples in the current batch
        _, predicted = torch.max(y_pred.data, 1)
        correct_examples += (predicted == targets).sum().item()
        total_examples += targets.size(0)
        ####################################

    avg_loss = train_loss / len(train_loader)
    avg_acc = correct_examples / total_examples
    print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))

    # add learning rate decay
    lrs.append(scheduler.get_last_lr())
    scheduler.step()  

    # Validate on the validation dataset
    #######################
    # your code here
    # switch to eval mode
    net.eval()
    #######################

    # this help you compute the validation accuracy
    total_examples = 0
    correct_examples = 0
    
    val_loss = 0 # again, track the validation loss if you want

    # disable gradient during validation, which can save GPU memory
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            ####################################
            # your code here
            # copy inputs to device
            inputs, targets = inputs.to(device), targets.to(device)
            
            # compute the output and loss
            y_pred = net(inputs)
            loss = criterion(y_pred, targets)
            val_loss += loss.item()
            
            # count the number of correctly predicted samples in the current batch
            _, predicted = torch.max(y_pred.data, 1)
            correct_examples += (predicted == targets).sum().item()
            total_examples += targets.size(0)
            ####################################

    avg_loss = val_loss / len(val_loader)
    avg_acc = correct_examples / total_examples
    print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))
    

    # save the model checkpoint
    if avg_acc > best_val_acc:
        best_val_acc = avg_acc
        if not os.path.exists(CHECKPOINT_FOLDER):
            os.makedirs(CHECKPOINT_FOLDER)
        print("Saving ...")
        state = {'state_dict': net.state_dict(),
                 'epoch': i,
                 'lr': scheduler.get_last_lr()}
        torch.save(state, os.path.join(CHECKPOINT_FOLDER, 'new_resnet_decay.pth'))
        
    print('')

print("="*50)
print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")

==> Training starts!
Epoch 0: LR: [0.1]


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:419: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Training loss: 1.7122, Training accuracy: 0.3631
Validation loss: 1.6466, Validation accuracy: 0.4348
Saving ...

Epoch 1: LR: [0.1]
Training loss: 1.2310, Training accuracy: 0.5549
Validation loss: 1.3883, Validation accuracy: 0.5440
Saving ...

Epoch 2: LR: [0.1]
Training loss: 1.0079, Training accuracy: 0.6413
Validation loss: 1.0702, Validation accuracy: 0.6240
Saving ...

Epoch 3: LR: [0.1]
Training loss: 0.8570, Training accuracy: 0.6976
Validation loss: 1.0112, Validation accuracy: 0.6480
Saving ...

Epoch 4: LR: [0.1]
Training loss: 0.7574, Training accuracy: 0.7363
Validation loss: 1.2355, Validation accuracy: 0.6000

Epoch 5: LR: [0.1]
Training loss: 0.6968, Training accuracy: 0.7587
Validation loss: 1.1384, Validation accuracy: 0.6606
Saving ...

Epoch 6: LR: [0.1]
Training loss: 0.6488, Training accuracy: 0.7769
Validation loss: 1.0799, Validation accuracy: 0.6522

Epoch 7: LR: [0.1]
Training loss: 0.6187, Training accuracy: 0.7858
Validation loss: 1.1993, Validation accura

In [17]:
# Test model on test set
# initialize a resnet and load trained weights
net = ResNet()
state_dict = torch.load(os.path.join(CHECKPOINT_FOLDER, 'new_resnet_decay.pth'))
net.load_state_dict(state_dict['state_dict'])
net.cuda()

# remember to switch to eval mode whenever you are making inference
net.eval()

results = []
with torch.no_grad():
    for x in test_loader:
        results.append(
            net(x.cuda()).argmax(1)
        )

# convert results to numpy array
results = torch.cat(results).cpu().numpy()
assert len(results) == 10000

#########################################################
with open('predictions2.csv', 'w') as fp:
    fp.write("Id,Label\n")
    for i in range(len(results)):
        fp.write("%d,%d\n" %(i, results[i]))